In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
import joblib

In [2]:
device = 'cuda'
num_workers=0
batch_size=10

transform=transforms.Compose([
#                             transforms.Grayscale(num_output_channels=1),
                            transforms.Resize((224,224)),
                            transforms.ToTensor()])

train_data=datasets.ImageFolder('../dataset/PlantVillage/train/',transform=transform)
# train_data=datasets.ImageFolder('../dataset/plantdisease/PlantVillage/',transform=transform)
valid_data = datasets.ImageFolder('../dataset/PlantVillage/val/',transform=transform)
test_data = datasets.ImageFolder('../dataset/test/',transform=transform)

train_loader=DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
valid_loader=DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers)
test_loader=DataLoader(test_data, batch_size=batch_size,num_workers=num_workers)

### Visualizing a batch of training data

In [3]:
# # dataiter= iter
# images, labels= next(iter(train_loader))
# images= images.numpy()

# fig=plt.figure(figsize=(25,4))

# for idx in np.arange(batch_size):
#     ax=fig.add_subplot(2, batch_size/2, idx+1, xticks=[], yticks=[])
# #     ax.imshow(np.squeeze(images[idx]), cmap='gray')
# #     ax.imshow(images[idx], cmap='gray')
#     plt.imshow(images[idx].transpose())
    
#     ax.set_title(str(labels[idx].item()))

# print(idx, labels[idx],labels[idx].item())
# print(labels)

In [4]:
# # dataiter= iter
# images, labels= next(iter(valid_loader))
# images= images.numpy()

# fig=plt.figure(figsize=(25,4))

# for idx in np.arange(batch_size):
#     ax=fig.add_subplot(2, batch_size/2, idx+1, xticks=[], yticks=[])
# #     ax.imshow(np.squeeze(images[idx]), cmap='gray')
#     ax.imshow(images[idx].transpose())
    
#     ax.set_title(str(labels[idx].item()))

# print(idx, labels[idx],labels[idx].item())
# print(labels)

### Architecture

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        n_classes = 39
        
        hidden_1 = 512
        hidden_2 = 512
        
        self.fc1 = nn.Linear(1024, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, n_classes)
        
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

model = models.googlenet(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
model.fc = Net()
model.to(device)
print(model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [7]:
n_epochs=2

valid_loss_min=np.Inf

for epoch in range(n_epochs):
    train_loss=0
    valid_loss=0
    
    model.train()
    for images, labels in train_loader:
        
        images, labels= images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output=model(images)
        loss=criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        train_loss+=loss.item() * images.size(0)
        
    model.eval()
    for images, labels in valid_loader:
        
        images, labels= images.to(device), labels.to(device)
        output=model(images)
        loss=criterion(output, labels)
        
        valid_loss+= loss.item() * images.size(0)
        
    train_loss /= len(train_loader)
    valid_loss /= len(valid_loader)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))

RuntimeError: CUDA error: device-side assert triggered

In [ ]:
n_epochs=20

valid_loss_min=np.Inf
train_loss_plt=[]
valid_loss_plt = []
valid_accuracy_plt=[]

for epoch in range(n_epochs):
    train_loss=0
    valid_loss=0
    
    model.train()
    for images, labels in train_loader:
        
        images, labels= images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output=model(images)
        loss=criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        train_loss+=loss.item() * images.size(0)
        
    model.eval()
    for images, labels in valid_loader:
        
        images, labels= images.cuda(), labels.cuda()
        output=model(images)
        loss=criterion(output, labels)
        valid_loss+= loss.item() * images.size(0)
        
    train_loss /= len(train_loader)
    valid_loss /= len(valid_loader)
    
    train_loss_plt.append(train_loss)
    valid_loss_plt.append(valid_loss)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))

### plots

In [ ]:
def plot(values, title, xaxis, yaxis,color):
    plt.plot(np.arange(len(values)),values, color=color)
    plt.title(title)
    plt.xlabel(xaxis)
    plt.ylabel(yaxis)
    plt.show()

In [ ]:
plot(train_loss_list, 'Train Loss', 'Epochs', 'CE Loss', 'b')
plot(valid_loss_list, 'Validation Loss', 'Epochs', 'CE Loss', 'r')

### Saving Model`

In [ ]:
model.load_state_dict(torch.load('GoogLeNet_model_weights.pt'))
filename = 'GoogLeNet_model.sav'
joblib.save(model, filename)

### Test the trained n/w

In [ ]:
test_loss = 0.0
n_classes = 39
class_correct = list(0. for i in range(n_classes))
class_total = list(0. for i in range(n_classes))

model.eval()
for data, target in test_loader:
    data, target= data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))